In [275]:
import joblib
import copy
from collections import Counter
import pandas as pd
import numpy as np
import os
import pickle
from xgboost import XGBRegressor
from catboost import Pool, CatBoostRegressor
from smt.surrogate_models import RBF,KRG,LS
from sklearn.feature_selection import f_regression
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn import svm
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel, Matern
from sklearn.gaussian_process import kernels
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split, KFold,GridSearchCV
from sklearn import preprocessing
from sklearn.feature_selection import VarianceThreshold
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.model_selection import learning_curve
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR,NuSVR
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
import plotly.colors as colors
import plotly.express as px
from plotly.subplots import make_subplots
from pymoo.algorithms.soo.nonconvex.cmaes import CMAES
from pymoo.termination import get_termination
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.core.problem import Problem
from pymoo.optimize import minimize
from pymoo.util.normalization import denormalize
from pymoo.visualization.scatter import Scatter
from smt.surrogate_models import RBF,KRG,LS
from scipy.interpolate import RBFInterpolator
# from smt.surrogate_models.rbf import RBF
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM
from pymoo.operators.sampling.rnd import FloatRandomSampling
from pymoo.algorithms.soo.nonconvex.de import DE
from pymoo.problems import get_problem
from pymoo.operators.sampling.lhs import LHS
from pymoo.optimize import minimize
from pymoo.algorithms.soo.nonconvex.isres import ISRES
import warnings
import joblib
import seaborn as sns
warnings.filterwarnings("ignore")
np.set_printoptions(suppress = True)

In [276]:
def getData():
    '''读取数据'''
    data1 = pd.read_csv('./data/data_1110.csv', na_values={'dist_out_in':0,'dist_out_mid':0,'dist_mid_in':0,'dist_in_hard':0})
    data2 = pd.read_csv('./data/data_1205.csv',na_values={'dist_out_in':0,'dist_out_mid':0,'dist_mid_in':0,'dist_in_hard':0})
    data = pd.concat([data1,data2], axis=0).iloc[:,1:].reset_index(drop=True)
    return data.iloc[:,:-1], data['HIC15']

In [277]:
rawData, rawLabel = getData()

# 数据预处理

## 删除缺失率大于80%的数据列

In [278]:
(rawData.isnull().sum()/rawData.shape[0]).sort_values(ascending=False)[:5]

dist_mid_in     0.811798
dist_out_mid    0.806180
dist_in_hard    0.351124
dist_out_in     0.199438
T_out           0.000000
dtype: float64

In [279]:
rawData.drop(columns=['dist_mid_in','dist_out_mid'], inplace=True)

In [280]:
rawData.shape

(356, 56)

## 均值填充缺失处理和方差筛选

In [281]:
def imputerVar(data, threshold=1):
    '''缺失处理和方差筛选'''
    simpleImputer = SimpleImputer().fit(data)
    joblib.dump(simpleImputer, './models/simpleImputer.pkl')
    siData = simpleImputer.transform(data)
    siData = pd.DataFrame(siData,columns=data.columns)
    varModel = VarianceThreshold(threshold=threshold).fit(siData)
#     print(data.shape)
    print(set(varModel.feature_names_in_)-set(varModel.get_feature_names_out()))
    siData = varModel.transform(siData)
    siData = pd.DataFrame(siData, columns=varModel.get_feature_names_out())
    inFeatures = siData.columns
    return siData, inFeatures

In [282]:
rawData, inFeatures = imputerVar(rawData, 0)

{'JL_X', 'T_S_mid_R', 'T_in', 'Y_S_out', 'head_V', 'Y_S_mid_F', 'delta_mid_R', 'T_S_mid_F', 'T_S_out', 'JR_Y', 'T_mid_R', 'delta_mid_F', 'Y_S_in', 'JR_Z', 'T_S_in', 'JR_X', 'Y_S_mid_R', 'JL_Y', 'delta_out', 'T_mid_F', 'T_out', 'delta_in'}


In [283]:
rawData.shape

(356, 34)

## 异常值检测

In [284]:
def solveoutliers(isExec, data, label, contamination):
    '''是否处理异常值'''
    def isoForest(data, label, flag, contamination=0.1):
        '''孤立森林去除异常值'''
        isoData = copy.deepcopy(data)
        clf = IsolationForest(max_samples='auto', random_state=0, max_features=1,contamination=contamination).fit(isoData)
        myIsoDataIndex = clf.predict(isoData)
        print(Counter(myIsoDataIndex))
#         print(myIsoDataIndex)
        data_ = isoData[myIsoDataIndex==1]
        label_ = label[myIsoDataIndex==1]
        isoData = isoData[myIsoDataIndex==-1]
        isoLabel = label[myIsoDataIndex==-1]
        pd.concat([isoData,isoLabel], axis=1,ignore_index=True).to_csv('./data/significantSample.csv')
        return data_, label_
    if isExec:
        return isoForest(data,label, contamination)
    return data, label

In [285]:
data, label = solveoutliers(False, rawData, rawLabel, 0.05)

In [286]:
data.columns

Index(['point_X', 'point_Y', 'point_Z', 'bonnet_YL', 'bonnet_YR', 'bonnet_XF',
       'bonnet_XR', 'bonnet_A', 'dist_out_in', 'dist_in_hard', 'Ixx', 'Iyy',
       'Izz', 'head_A', 'head_M', 'JL_Z', 'LL_X', 'LL_Y', 'LL_Z', 'LR_X',
       'LR_Y', 'LR_Z', 'B_LR_X', 'B_LR_Y', 'B_LR_Z', 'B_LF_X', 'B_LF_Y',
       'B_LF_Z', 'B_RR_X', 'B_RR_Y', 'B_RR_Z', 'B_RF_X', 'B_RF_Y', 'B_RF_Z'],
      dtype='object')

In [287]:
label.shape

(356,)

# 数据标准化

In [288]:
def stdModel(data):
    '''数据标准化，然后拆分数据集'''
    stdModel = preprocessing.StandardScaler().fit(data)
    stdData = stdModel.transform(data)
    joblib.dump(stdModel,'./models/stdModel.pkl')
    data = pd.DataFrame(stdData, columns=data.columns)
    return data
# data = stdModel(data)

In [289]:
data.shape

(356, 34)

In [290]:
colNames = data.columns
colNames

Index(['point_X', 'point_Y', 'point_Z', 'bonnet_YL', 'bonnet_YR', 'bonnet_XF',
       'bonnet_XR', 'bonnet_A', 'dist_out_in', 'dist_in_hard', 'Ixx', 'Iyy',
       'Izz', 'head_A', 'head_M', 'JL_Z', 'LL_X', 'LL_Y', 'LL_Z', 'LR_X',
       'LR_Y', 'LR_Z', 'B_LR_X', 'B_LR_Y', 'B_LR_Z', 'B_LF_X', 'B_LF_Y',
       'B_LF_Z', 'B_RR_X', 'B_RR_Y', 'B_RR_Z', 'B_RF_X', 'B_RF_Y', 'B_RF_Z'],
      dtype='object')

# 数据拆分

In [291]:
def splitData(flag, label):
    '''数据拆分'''
#     data['label'] = rawLabel
    xTrain,xTest,yTrain,yTest = train_test_split(data,label,test_size=0.1,random_state=420)
    return xTrain,xTest,yTrain,yTest

In [292]:
xTrain,xTest,yTrain,yTest = splitData(data, label)

In [293]:
xTrain.shape

(320, 34)

In [294]:
xTest.shape

(36, 34)

In [295]:
xTest.columns

Index(['point_X', 'point_Y', 'point_Z', 'bonnet_YL', 'bonnet_YR', 'bonnet_XF',
       'bonnet_XR', 'bonnet_A', 'dist_out_in', 'dist_in_hard', 'Ixx', 'Iyy',
       'Izz', 'head_A', 'head_M', 'JL_Z', 'LL_X', 'LL_Y', 'LL_Z', 'LR_X',
       'LR_Y', 'LR_Z', 'B_LR_X', 'B_LR_Y', 'B_LR_Z', 'B_LF_X', 'B_LF_Y',
       'B_LF_Z', 'B_RR_X', 'B_RR_Y', 'B_RR_Z', 'B_RF_X', 'B_RF_Y', 'B_RF_Z'],
      dtype='object')

# 创建模型

## 线性回归模型

In [296]:
def createLrModelSklearn():
    model = LinearRegression()
    model.fit(xTrain, yTrain)
    yPredTest = model.predict(xTest)
    yPredTrain = model.predict(xTrain)
    r2Test = r2_score(yTest, yPredTest).round(3)
    r2Train = r2_score(yTrain, yPredTrain).round(3)
    maeTest = mean_absolute_error(yTest, yPredTest).round(3)
    maeTrain = mean_absolute_error(yTrain, yPredTrain).round(3)
    print(f'modelname:{model},\ntrainR2:{r2Train},\ntestR2:{r2Test}')
    print(model.coef_)
    joblib.dump(model, './models/lrModelSklearn.pkl')
    return maeTest, maeTrain,r2Train, r2Test, model, yPredTest

In [297]:
lrMaeTestSklearn, lrMaeTrainSklearn, lrR2TrainSklearn, lrR2TestSklearn, lrModelSklearn, lryPredTestSklearn = createLrModelSklearn()

modelname:LinearRegression(),
trainR2:0.849,
testR2:0.845
[ 1.91260510e+00 -2.48904533e-01 -2.26529386e+01 -7.54241606e-03
  2.81824783e-02 -2.02609330e-02 -6.84957096e-02 -7.49758190e+00
 -1.69303263e+00 -8.36350074e-01  1.41466335e+02  1.30844493e+02
 -1.37909776e+02  1.68671438e+01  6.65590541e+01  6.12633090e+07
  3.34800720e+09  2.42871445e+10  9.56007799e+08  3.34800720e+09
 -6.27086807e+09  9.56007799e+08 -4.67895605e+09 -3.00210567e+10
  6.88860830e+08 -8.68685931e+09  2.00946867e+10  1.10670585e+09
  8.53983176e+09  6.59006008e+09  4.34800467e+08 -4.43156739e+09
 -3.21142654e+09  7.36042467e+08]


# 支持向量机

In [300]:
def createSVRModelSklearn():
    '''创建支持向量机回归模型'''
    model = SVR()
    model.fit(xTrain, yTrain)
    yPredTest = model.predict(xTest)
    yPredTrain = model.predict(xTrain)
    r2Test = r2_score(yTest, yPredTest).round(3)
    r2Train = r2_score(yTrain, yPredTrain).round(3)
    maeTest = mean_absolute_error(yTest, yPredTest).round(3)
    maeTrain = mean_absolute_error(yTrain, yPredTrain).round(3)
    print(f'modelname:{model},\ntrainR2:{r2Train},\ntestR2:{r2Test}')
    joblib.dump(model, './models/svrModelSklearn.pkl')
    return maeTest, maeTrain,r2Train, r2Test, model, yPredTest

In [301]:
svrMaeTestSklearn, svrMaeTrainSklearn, svrR2TrainSklearn, svrR2TestSklearn, svrModelSklearn, svryPredTestSklearn = createSVRModelSklearn()

modelname:SVR(),
trainR2:-0.1,
testR2:-0.094


## 特征重要性函数编写

In [302]:
def getFeaturesImportance(model, n, flag, colNames):
    '''获取特征重要性排序,model:模型名字，n：过滤阈值，flag:模型名字，colNames，列名字'''
    randomForestFeaturesImportance = dict(zip(colNames, model.feature_importances_))
    sorted(randomForestFeaturesImportance.items(), key=lambda d: d[1], reverse=True)
    randomForestFeaturesImportance = pd.DataFrame.from_dict(randomForestFeaturesImportance,orient='index') \
                                                            .rename(columns={'index':'featureName',0:'score'}) \
                                                            .sort_values(by='score', ascending=False)
    randomForestFeaturesImportance = randomForestFeaturesImportance[randomForestFeaturesImportance['score']>n].reset_index()
    randomForestFeaturesImportance.to_csv('./data/'+flag+'FeaturesImportance.csv', index=False)
    return randomForestFeaturesImportance

## 随机森林

In [303]:
def createRandomForestModelSklearn():
    '''创建随机森林模型'''
    model = RandomForestRegressor(random_state=1)
    model.fit(xTrain, yTrain)
    yPredTest = model.predict(xTest)
    yPredTrain = model.predict(xTrain)
    r2Test = r2_score(yTest, yPredTest).round(3)
    r2Train = r2_score(yTrain, yPredTrain).round(3)
    maeTest = mean_absolute_error(yTest, yPredTest).round(3)
    maeTrain = mean_absolute_error(yTrain, yPredTrain).round(3)
    print(f'modelname:{model},\ntrainR2:{r2Train},\ntestR2:{r2Test}')
    joblib.dump(model, './models/randomForestModelSklearn.pkl')
    return maeTest, maeTrain,r2Train, r2Test, model, yPredTest


In [304]:
randomForestMaeTestSklearn, randomForestMaeTrainSklearn, randomForestR2TrainSklearn, randomForestR2TestSklearn, randomForestModelSklearn, randomForestyPredTestSklearn = createRandomForestModelSklearn()


modelname:RandomForestRegressor(random_state=1),
trainR2:0.98,
testR2:0.909


In [305]:
randomForestFeaturesImportance = getFeaturesImportance(randomForestModelSklearn,0.,'randomForest',colNames)


## XGB模型

In [306]:
def createXgbModelSklearn():
    '''创建XGB模型'''
    model = XGBRegressor()
    model.fit(xTrain, yTrain)
    yPredTest = model.predict(xTest)
    yPredTrain = model.predict(xTrain)
    r2Test = r2_score(yTest, yPredTest).round(3)
    r2Train = r2_score(yTrain, yPredTrain).round(3)
    maeTest = mean_absolute_error(yTest, yPredTest).round(3)
    maeTrain = mean_absolute_error(yTrain, yPredTrain).round(3)
    print(f'modelname:{model},\ntrainR2:{r2Train},\ntestR2:{r2Test}')
    joblib.dump(model, './models/xgbModelSklearn.pkl')
    return maeTest, maeTrain,r2Train, r2Test, model, yPredTest

In [307]:
xgbMaeTestSklearn, xgbMaeTrainSklearn, xgbR2TrainSklearn, xgbR2TestSklearn, xgbModelSklearn, xgbyPredTestSklearn =createXgbModelSklearn()

modelname:XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...),
trainR2:1.0,
testR2:0.933


In [308]:
randomForestFeaturesImportance = getFeaturesImportance(xgbModelSklearn,0.,'xgbModel',colNames)


## CatBoostRegressor

In [309]:
def createCatBoostModel():
    '''创建CatBoost模型'''
    # initialize Pool
    trainPool = Pool(xTrain,
                      yTrain)
    testPool = Pool(xTest) 

    # specify the training parameters 
    model = CatBoostRegressor(silent=True)
    #train the model
    model.fit(trainPool)
    # make the prediction using the resulting model
    yPredTest = model.predict(xTest)
    yPredTrain = model.predict(xTrain)
    r2Test = r2_score(yTest, yPredTest).round(3)
    r2Train = r2_score(yTrain, yPredTrain).round(3)
    maeTest = mean_absolute_error(yTest, yPredTest).round(3)
    maeTrain = mean_absolute_error(yTrain, yPredTrain).round(3)
    print(f'modelname:{model},\ntrainR2:{r2Train},\ntestR2:{r2Test}')
    joblib.dump(model, './models/catBoostModel.pkl')
    return maeTest, maeTrain,r2Train, r2Test, model, yPredTest


In [310]:
catBoostMaeTestSklearn, catBoostMaeTrainSklearn, catBoostR2TrainSklearn, catBoostR2TestSklearn, catBoostModelSklearn, catBoostyPredTestSklearn = createCatBoostModel()

modelname:<catboost.core.CatBoostRegressor object at 0x000001FECE470150>,
trainR2:0.999,
testR2:0.917


In [311]:
catBoostFeaturesImportance = getFeaturesImportance(catBoostModelSklearn,0.,'catBoost',colNames)


In [312]:
colNames

Index(['point_X', 'point_Y', 'point_Z', 'bonnet_YL', 'bonnet_YR', 'bonnet_XF',
       'bonnet_XR', 'bonnet_A', 'dist_out_in', 'dist_in_hard', 'Ixx', 'Iyy',
       'Izz', 'head_A', 'head_M', 'JL_Z', 'LL_X', 'LL_Y', 'LL_Z', 'LR_X',
       'LR_Y', 'LR_Z', 'B_LR_X', 'B_LR_Y', 'B_LR_Z', 'B_LF_X', 'B_LF_Y',
       'B_LF_Z', 'B_RR_X', 'B_RR_Y', 'B_RR_Z', 'B_RF_X', 'B_RF_Y', 'B_RF_Z'],
      dtype='object')

# 加载所有模型并预测

In [313]:
# 分别从F3取一个值，F2车型取两个值，用于测试
predData = pd.read_csv('./data/preData2.csv')[colNames].values
# 取F2数据预测
# yPredData = pd.read_csv('./data/data_0104.csv', na_values={'dist_out_in':-1,'dist_out_mid':-1,'dist_mid_in':-1,'dist_in_hard':-1}).iloc[:,1:]['HIC15'].values
# predData = pd.read_csv('./data/data_0104.csv', na_values={'dist_out_in':-1,'dist_out_mid':-1,'dist_mid_in':-1,'dist_in_hard':-1}).iloc[:,1:][colNames]
# predData = SimpleImputer().fit_transform(predData)

In [315]:
def loadModelAndPred(data,yTrue=None):
    '''加载预测模型进行验证'''
    lryPredSklearn = joblib.load('./models/lrModelSklearn.pkl').predict(data)
    randomForestyPredSklearn = joblib.load('./models/randomForestModelSklearn.pkl').predict(data)
    svryPredSklearn = joblib.load('./models/svrModelSklearn.pkl').predict(data)
    xgbyPredSklearn = joblib.load('./models/xgbModelSklearn.pkl').predict(data)
    catBoostyPred = joblib.load('./models/catBoostModel.pkl').predict(data)
    # 收集预测结果形成csv文件
    allModelPreds = {
                    '线性回归':lryPredSklearn,
                    '随机森林':randomForestyPredSklearn,
                    '支持向量机':svryPredSklearn,
                    'xgboost':xgbyPredSklearn,
                    'catBoost':catBoostyPred
    }
    allModelPreds = pd.DataFrame(allModelPreds).round(3)
    if yTrue is not None:
        allModelPreds['真实值'] = yTrue
        #  allModelPredsF2F3
#         allModelPreds.to_csv('./data/allModelPredsF3.csv', encoding='gbk')
        allModelPreds.to_csv('./data/allModelPredsF2F3.csv', encoding='gbk')
    return allModelPreds
# 
# allModelPreds = loadModelAndPred(predData)
allModelPreds = loadModelAndPred(xTest.values,yTrue=yTest.values)
allModelPreds.head()

,线性回归,随机森林,支持向量机,xgboost,catBoost,真实值
0,364.977,486.852,730.627,483.415009,490.890,493.92
1,672.773,592.661,732.302,533.653992,601.623,635.87
2,756.133,816.896,732.339,748.197998,791.199,586.61
3,846.945,976.432,732.922,926.643982,926.366,1215.94
4,415.453,555.304,730.775,565.078003,562.696,604.13


In [316]:
r2_score(allModelPreds['真实值'], allModelPreds['catBoost'])

0.9166180614791808

In [317]:
r2_score(allModelPreds['真实值'], allModelPreds['catBoost'])

0.9166180614791808

# 求百分比1-abs(yTrue-yTest)/yTrue

In [318]:
allModelPredsPercentage = pd.DataFrame()
allModelPredsPercentage['xgboostPercent'] = 1- (allModelPreds['真实值'] - allModelPreds['xgboost']).abs()/allModelPreds['真实值']
allModelPredsPercentage['catboostPercent'] = 1- (allModelPreds['真实值'] - allModelPreds['catBoost']).abs()/allModelPreds['真实值']
allModelPredsPercentage.to_csv('./data/allModelPredsPercentF3.csv', encoding='gbk')

## 求所有的R2数据

In [320]:

def createAllModel():
    '''集中统一调用,
    线性回归模型	支持向量机模型	随机森林模型	XGB模型-->catBoost模型
    '''
    print('*'*90)
    lrMaeTestSklearn, lrMaeTrainSklearn, lrR2TrainSklearn, lrR2TestSklearn, lrModelSklearn, lryPredTestSklearn = createLrModelSklearn()
    print('*'*90)
    svrMaeTestSklearn, svrMaeTrainSklearn, svrR2TrainSklearn, svrR2TestSklearn, svrModelSklearn, svryPredTestSklearn = createSVRModelSklearn()
    print('*'*90)
    randomForestMaeTestSklearn, randomForestMaeTrainSklearn, randomForestR2TrainSklearn, randomForestR2TestSklearn, randomForestModelSklearn, randomForestyPredTestSklearn = createRandomForestModelSklearn()
    print('*'*90)
    xgbMaeTestSklearn, xgbMaeTrainSklearn, xgbR2TrainSklearn, xgbR2TestSklearn, xgbModelSklearn, xgbyPredTestSklearn =createXgbModelSklearn()
    print('*'*90)
    catBoostMaeTestSklearn, catBoostMaeTrainSklearn, catBoostR2TrainSklearn, catBoostR2TestSklearn, catBoostModelSklearn, catBoostyPredTestSklearn = createCatBoostModel()    
    allModelR2 = {
                    '线性回归':[lrR2TrainSklearn, lrR2TestSklearn],
                    '支持向量机':[svrR2TrainSklearn, svrR2TestSklearn],
                    '随机森林':[randomForestR2TrainSklearn, randomForestR2TestSklearn],
                    'xgboost':[xgbR2TrainSklearn, xgbR2TestSklearn],
                    'catBoost':[catBoostR2TrainSklearn, catBoostR2TestSklearn]
    }
    allModelR2 = pd.DataFrame(allModelR2,index=['训练集', '测试集'])
    allModelR2.to_csv('./data/allModelR2F3.csv', encoding='gbk')
    allModelMae = {
                    '线性回归':[lrMaeTrainSklearn, lrMaeTestSklearn],
                    '支持向量机':[svrMaeTrainSklearn, svrMaeTestSklearn],
                    '随机森林':[randomForestMaeTrainSklearn, randomForestMaeTestSklearn],
                    'xgboost':[xgbMaeTrainSklearn, xgbMaeTestSklearn],
                    'catBoost':[catBoostMaeTrainSklearn, catBoostMaeTestSklearn]
    }
    allModelMae = pd.DataFrame(allModelMae,index=['训练集', '测试集'])
    allModelMae.to_csv('./data/allModelMaeF3.csv', encoding='gbk')
    
    return allModelR2, allModelMae
createAllModel()    
   

******************************************************************************************
modelname:LinearRegression(),
trainR2:0.849,
testR2:0.845
[ 1.91260510e+00 -2.48904533e-01 -2.26529386e+01 -7.54241606e-03
  2.81824783e-02 -2.02609330e-02 -6.84957096e-02 -7.49758190e+00
 -1.69303263e+00 -8.36350074e-01  1.41466335e+02  1.30844493e+02
 -1.37909776e+02  1.68671438e+01  6.65590541e+01  6.12633090e+07
  3.34800720e+09  2.42871445e+10  9.56007799e+08  3.34800720e+09
 -6.27086807e+09  9.56007799e+08 -4.67895605e+09 -3.00210567e+10
  6.88860830e+08 -8.68685931e+09  2.00946867e+10  1.10670585e+09
  8.53983176e+09  6.59006008e+09  4.34800467e+08 -4.43156739e+09
 -3.21142654e+09  7.36042467e+08]
******************************************************************************************
modelname:SVR(),
trainR2:-0.1,
testR2:-0.094
******************************************************************************************
modelname:RandomForestRegressor(random_state=1),
trainR2:0.98,
testR2:

(      线性回归  支持向量机   随机森林  xgboost  catBoost
 训练集  0.849 -0.100  0.980    1.000     0.999
 测试集  0.845 -0.094  0.909    0.933     0.917,
         线性回归    支持向量机    随机森林  xgboost  catBoost
 训练集  129.955  320.261  34.725    0.116     9.089
 测试集  131.690  288.860  89.173   71.550    79.820)

 ## 汇总模型所有特征重要性

In [90]:
def getAllFeaturesImportance(colNames):
    '''加载预测模型进行验证'''
    randomForestyPredSklearn = joblib.load('./models/randomForestModelSklearn.pkl')
    xgbyPredSklearn = joblib.load('./models/xgbModelSklearn.pkl')
    catBoostyPred = joblib.load('./models/catBoostModel.pkl')
    # 汇总模型所有特征重要性
    randomForestFeaturesImportance = getFeaturesImportance(randomForestyPredSklearn, -1.,'randomForest',colNames)
    xgbFeaturesImportance = getFeaturesImportance(xgbyPredSklearn, -1.,'xgboost',colNames)
    catBoostFeaturesImportance = getFeaturesImportance(catBoostyPred, -1.,'catBoost',colNames)
    allFeaturesImportance = pd.concat([randomForestFeaturesImportance,xgbFeaturesImportance,catBoostFeaturesImportance], axis=1)
    return allFeaturesImportance

In [91]:
getAllFeaturesImportance(colNames)

,index,score,index,score,index,score
0,point_Z,0.321181,Izz,0.330093,point_Z,22.675357
1,dist_in_hard,0.248033,point_Y,0.222342,dist_in_hard,21.198498
2,dist_out_in,0.123188,dist_out_in,0.145209,dist_out_in,9.213565
3,bonnet_YR,0.051920,Ixx,0.079614,point_Y,6.551755
4,bonnet_A,0.039960,bonnet_A,0.061903,bonnet_A,6.447501
5,Iyy,0.039758,Iyy,0.061554,bonnet_YL,5.751738
6,Izz,0.036942,point_X,0.037464,Ixx,5.427982
7,bonnet_YL,0.035193,dist_in_hard,0.020897,bonnet_YR,4.060978
8,bonnet_XR,0.033523,bonnet_XR,0.007697,Iyy,3.994834
9,point_Y,0.030294,bonnet_YL,0.007635,bonnet_XF,3.098667


# Optuna模型调参

In [484]:
import optuna
import xgboost as xgb
import sklearn
import lightgbm as lgb
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_rank
from optuna.visualization import plot_slice
from optuna.visualization import plot_timeline

In [485]:
SEED = 42
np.random.seed(SEED)

In [486]:
def createStudy(Objective, studyName):
    study = optuna.create_study(
        direction="minimize",
        sampler=optuna.samplers.TPESampler(seed=SEED),
        pruner=optuna.pruners.MedianPruner(n_warmup_steps=10),
        storage="sqlite:///db.sqlite3",  # Specify the storage URL here.
        study_name=studyName,
        load_if_exists=True
    )
    study.optimize(Objective, n_trials=10, timeout=600)
    return study

In [487]:
def lgbObjective(trial):
    dtrain = lgb.Dataset(xTrain, label=yTrain)
    dvalid = lgb.Dataset(xTest, label=yTest)

    param = {
        "objective": "regression",
        "metric": "rmse",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "num_iterations":trial.suggest_int('num_iterations', 5, 20),
        "learning_rate":trial.suggest_float("learning_rate",0.01, 0.1, step=0.01)
    }
    
    # Add a callback for pruning.
    pruning_callback = optuna.integration.LightGBMPruningCallback(trial, "rmse")
    gbm = lgb.train(param, dtrain, valid_sets=[dvalid], callbacks=[pruning_callback])
    
    preds = gbm.predict(xTest)
#     pred_labels = np.rint(preds)
    accuracy = sklearn.metrics.r2_score(yTest, preds)
    return accuracy

In [488]:
lgbStudy = createStudy(lgbObjective, 'lgbModel')

[I 2024-01-16 09:18:32,242] Using an existing study with name 'lgbModel' instead of creating a new one.
[I 2024-01-16 09:18:32,770] Trial 62 finished with value: -0.09660984226137215 and parameters: {'num_iterations': 8, 'learning_rate': 0.02}. Best is trial 23 with value: -0.3130250037724056.
[I 2024-01-16 09:18:33,108] Trial 63 finished with value: -0.05880308651323385 and parameters: {'num_iterations': 5, 'learning_rate': 0.04}. Best is trial 23 with value: -0.3130250037724056.
[I 2024-01-16 09:18:33,447] Trial 64 finished with value: -0.0787230699653596 and parameters: {'num_iterations': 6, 'learning_rate': 0.03}. Best is trial 23 with value: -0.3130250037724056.
[I 2024-01-16 09:18:33,922] Trial 65 pruned. Trial was pruned at iteration 10.
[I 2024-01-16 09:18:34,247] Trial 66 finished with value: -0.16279747593898852 and parameters: {'num_iterations': 5, 'learning_rate': 0.02}. Best is trial 23 with value: -0.3130250037724056.
[I 2024-01-16 09:18:34,589] Trial 67 finished with val

In [489]:
lgbStudy.best_params

{'num_iterations': 6, 'learning_rate': 0.02}

In [490]:
def xgbObjective(trial):
#     data, target = sklearn.datasets.load_breast_cancer(return_X_y=True)
#     train_x, valid_x, train_y, valid_y = train_test_split(data, target, test_size=0.25)
    dtrain = xgb.DMatrix(xTrain, label=yTrain)
    dvalid = xgb.DMatrix(xTest, label=yTest)

    param = {
        "verbosity": 0,
        "objective": "reg:squarederror",
        "eval_metric": "rmse",
        "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
    }
    
    # Add a callback for pruning.
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "validation-rmse")
    bst = xgb.train(param, dtrain, evals=[(dvalid, "validation")], callbacks=[pruning_callback])
    
    preds = bst.predict(dvalid)
#     pred_labels = np.rint(preds)
    accuracy = sklearn.metrics.r2_score(yTest, preds)
    return accuracy

In [45]:
xgbStudy = createStudy(xgbObjective, 'xgbModel')

[I 2024-01-08 17:08:26,602] A new study created in RDB with name: xgbModel


[0]	validation-rmse:253.93806
[1]	validation-rmse:231.54088
[2]	validation-rmse:224.59290
[3]	validation-rmse:221.49101
[4]	validation-rmse:219.29585
[5]	validation-rmse:217.97031
[6]	validation-rmse:215.58869
[7]	validation-rmse:214.07178
[8]	validation-rmse:212.38127
[9]	validation-rmse:210.33935


[I 2024-01-08 17:08:27,224] Trial 0 finished with value: 0.6249155485128661 and parameters: {'booster': 'gblinear', 'lambda': 0.0006155564318973012, 'alpha': 1.77071686435378e-07}. Best is trial 0 with value: 0.6249155485128661.


[0]	validation-rmse:252.52479
[1]	validation-rmse:191.34167
[2]	validation-rmse:155.98354
[3]	validation-rmse:128.53757
[4]	validation-rmse:112.24430
[5]	validation-rmse:105.20590
[6]	validation-rmse:99.22480
[7]	validation-rmse:92.38507
[8]	validation-rmse:90.98924
[9]	validation-rmse:87.81237


[I 2024-01-08 17:08:27,830] Trial 1 finished with value: 0.9346268654214207 and parameters: {'booster': 'dart', 'lambda': 0.0006440507553993703, 'alpha': 0.004619347374377372}. Best is trial 0 with value: 0.6249155485128661.


...

[I 2024-01-08 17:08:31,491] Trial 7 finished with value: 0.9447585563895793 and parameters: {'booster': 'gbtree', 'lambda': 0.0029775853025212607, 'alpha': 3.320625892007924e-05}. Best is trial 0 with value: 0.6249155485128661.


[0]	validation-rmse:259.14529
[1]	validation-rmse:238.36115
[2]	validation-rmse:229.93109
[3]	validation-rmse:225.05806
[4]	validation-rmse:221.90234
[5]	validation-rmse:219.64807
[6]	validation-rmse:218.14746
[7]	validation-rmse:216.88437
[8]	validation-rmse:215.47117
[9]	validation-rmse:214.34255


[I 2024-01-08 17:08:32,120] Trial 8 finished with value: 0.6105024021138996 and parameters: {'booster': 'gblinear', 'lambda': 0.1881755597772026, 'alpha': 1.1755466083160747e-06}. Best is trial 8 with value: 0.6105024021138996.


[0]	validation-rmse:252.50752
[1]	validation-rmse:191.32033
[2]	validation-rmse:155.96714
[3]	validation-rmse:128.52672
[4]	validation-rmse:112.23790
[5]	validation-rmse:105.20272
[6]	validation-rmse:99.22513
[7]	validation-rmse:92.38742
[8]	validation-rmse:90.99309
[9]	validation-rmse:87.81888


[I 2024-01-08 17:08:32,822] Trial 9 finished with value: 0.9346171631018271 and parameters: {'booster': 'gbtree', 'lambda': 0.00023641892308789696, 'alpha': 3.0118659882617117e-07}. Best is trial 8 with value: 0.6105024021138996.


In [46]:
xgbStudy.best_params

{'booster': 'gblinear',
 'lambda': 0.1881755597772026,
 'alpha': 1.1755466083160747e-06}